In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    Src['Datetime']                     = Query['datetime']
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']   = (Src['Close'] - Src['Open'])
    Src['Hilo']     = (Src['High']  - Src['Low'])
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc[f'Open -{1}']  = Calc['Open']  .shift(+1)
    Calc[f'High -{1}']  = Calc['High']  .shift(+1)
    Calc[f'Low -{1}']   = Calc['Low']   .shift(+1)
    Calc[f'Close -{1}'] = Calc['Close'] .shift(+1)
    
    Calc[f'Hilo -{1}']  = (Calc[f'High -{1}'] - Calc[f'Low -{1}'])
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

In [ ]:
def SIGNALS(Calc):
    Sign = pd.DataFrame(Calc)

    for OHLC in ['Open','High','Low','Close']:
        Sign[f'{OHLC} out Max'] =                                                 (              Calc['High -1'] < Calc[OHLC])
        Sign[f'{OHLC} in Max']  =                                                 (Calc[OHLC] <= Calc['High -1']             )
        Sign[f'{OHLC} inside']  = (              Calc['Low -1'] < Calc[OHLC])  &  (Calc[OHLC] <= Calc['High -1']             )
        Sign[f'{OHLC} in Min']  = (              Calc['Low -1'] < Calc[OHLC])
        Sign[f'{OHLC} out Min'] = (Calc[OHLC] <= Calc['Low -1']             )

    Calc['Break 0'] = (Sign['Open in Min']  & Sign['Open in Max'])
    Calc['Break 1'] = (Sign['Open out Min'] | Sign['Open out Max'])  
    return Sign

Sign = SIGNALS(Calc)
Sign.head()

In [21]:
def STATS(Calc):
    BINS = [500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000]
    pipe = []
    for b in BINS:
        Df = Calc[Calc[f'Hilo {-1}'] > b] #.reset_index(drop=1)
        pipe.append({
            'Hilo D-1 >':   b,
            'Count':        len(Df),
            'Inside':       (Df['Break 0'].mean() *100).round(0).astype(int), 
            'Outside':      (Df['Break 1'].mean() *100).round(0).astype(int), 
        })
    pass
    return pd.DataFrame(pipe)

STATS(Calc)

,Hilo D-1 >,Count,Inside,Outside
0,500,1244,78,22
1,1000,1180,79,21
2,1500,863,79,21
3,2000,492,78,22
4,2500,258,80,20
5,3000,140,82,18
6,3500,59,83,17
7,4000,31,81,19
8,4500,15,93,7
9,5000,6,83,17


In [19]:
# Df = STATS(Calc)
# Df['Total'] = Df[['Inside','Outside']].sum(axis=1)
# Df

# Snippets

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc[f'Open -{1}']  = Calc['Open']  .shift(+1)
    Calc[f'High -{1}']  = Calc['High']  .shift(+1)
    Calc[f'Low -{1}']   = Calc['Low']   .shift(+1)
    Calc[f'Close -{1}'] = Calc['Close'] .shift(+1)
    
    Calc[f'Hilo -{1}']  = (Calc[f'High -{1}'] - Calc[f'Low -{1}'])
    return Calc


def SIGNALS(Calc):
    Sign = pd.DataFrame(Calc)

    for OHLC in ['Open','High','Low','Close']:
        Sign[f'{OHLC} out Max'] =                                                 (              Calc['High -1'] < Calc[OHLC])
        Sign[f'{OHLC} in Max']  =                                                 (Calc[OHLC] <= Calc['High -1']             )
        Sign[f'{OHLC} inside']  = (              Calc['Low -1'] < Calc[OHLC])  &  (Calc[OHLC] <= Calc['High -1']             )
        Sign[f'{OHLC} in Min']  = (              Calc['Low -1'] < Calc[OHLC])
        Sign[f'{OHLC} out Min'] = (Calc[OHLC] <= Calc['Low -1']             )

    Calc['Break 0'] = (Sign['Open in Min']  & Sign['Open in Max'])
    Calc['Break 1'] = (Sign['Open out Min'] | Sign['Open out Max'])  
    return Sign


def STATS(Calc):
    BINS = [500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000]
    pipe = []
    for b in BINS:
        Df = Calc[Calc[f'Hilo {-1}'] > b] #.reset_index(drop=1)
        pipe.append({
            'Hilo D-1 >':   b,
            'Count':        len(Df),
            'Inside':       (Df['Break 0'].mean() *100).round(0).astype(int), 
            'Outside':      (Df['Break 1'].mean() *100).round(0).astype(int), 
        })
    pass
    return pd.DataFrame(pipe)